- - -

### `Transformaciones`

Transformación de datos de los archivos de puntuaciones y plataformas como preparación y generación de los archivos necesarios como fuente para las operaciones de la API. 

In [66]:
import os
from datetime import datetime
from statistics import mode
import numpy as np
import pandas as pd

Función generadora de un archivo `csv` integrando los que encuentra en una ruta determinada.

In [67]:
def join_csv(path_in=None, path_out=None, out_file=None):
    """
    Combina los archivos CSV existentes en una carpeta y devuelve un DataFrame.
    
    Parameters
    ----------
    path : string
        Ruta en la que se encuentran los archivos, si no se indica ninguna se
        usará la carpeta actual.
    out_file : string
        Archivo opcional en el que se guardará los resultados, en caso de que no
        se indique un nombre no se generará.
    
    Returns
    -------
    DataFrame
        Objeto DataFrame con la unión de los archivos CSV
    int
        Número de renglones del Dataframe
    """
    
    if path_in is None:
        files = [file for file in os.listdir() if '.csv' in file]
    else:
        files = [os.path.join(path_in, file) for file in os.listdir(path_in) if '.csv' in file]
        
    df = pd.concat(map(pd.read_csv, files), ignore_index=True)
    
    if out_file is not None:
        if path_out is None:
            df.to_csv(out_file)
        else:
            df.to_csv(os.path.join(path_out, out_file), index=False)
        
    return df, len(df)

Genera un solo archivo de `ratings` y lo ajusta para integrarlo con el de `plataformas-titulos`

In [68]:
df_ratings, df_ratings_len = join_csv(path_in=r'D:\Henry\Repo\PP\datasets\ratings',
                                      path_out=r'D:\Henry\Repo\PP\datasets\ratings\join', 
                                      out_file='ratings.csv')

In [69]:
#df_ratings.drop(columns='Unnamed: 0', inplace=True)
df_ratings.rename({'rating' : 'score' , 'movieId' : 'id'}, axis= 1, inplace=True)
df_ratings['id'] = df_ratings['id'].astype('string')

df_ratings['score_date'] = pd.to_datetime(df_ratings['timestamp'], unit='s')
df_ratings['score_date'] = df_ratings['score_date'].astype('string')
df_ratings['score_date'] = df_ratings['score_date'].str.slice(stop=10)

df_ratings = pd.DataFrame.reindex(df_ratings, columns = ['id', 'userId', 'score', 'score_date'])

df_ratings.to_csv(r'D:\Henry\Repo\PP\datasets\ratings\join\ratings_fixed.csv', index=False)

In [70]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11024289 entries, 0 to 11024288
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   id          string 
 1   userId      int64  
 2   score       float64
 3   score_date  string 
dtypes: float64(1), int64(1), string(2)
memory usage: 336.4 MB


In [71]:
df_ratings.head()

,id,userId,score,score_date
0,as680,1,1.0,2015-03-09
1,ns2186,1,4.5,2015-03-09
2,hs2381,1,5.0,2015-03-09
3,ns3663,1,5.0,2015-03-09
4,as9500,1,5.0,2015-03-09


Función de transformación de un archivo `plataforma-titulos` para uso posterior.

In [72]:
def fix_platform_file(in_file, platform, out_file):
    """
    Ajusta la información del archivo de titulos y genera uno de salida.
    
    Parameters
    ----------
    in_file : string
        Archivo de entrada de titulos incluyendo la ruta completa.
    platform : string
        Identificador de la plataforma
    out_file : string
        Archivo de salida de titulos incluyendo la ruta completa.
    
    Returns
    -------
    DataFrame
        Objeto DataFrame con la estructura preparada para mezclar con 
        el archivo de ratings.
    """
    dat_types = {'show_id' : 'string', 
                 'type' : 'string',
                 'title' : 'string',
                 'director' : 'string',
                 'cast' : 'string',
                 'country' : 'string',
                 'date_added' : 'string',
                 'release_year' : 'int64',
                 'rating' : 'string',
                 'duration' : 'string',
                 'listed_in' : 'string',
                 'description' : 'string'}
    
    df = pd.read_csv(in_file, dtype=dat_types)

    """
    Genera el campo 'id' compuesto de la primera letra del nombre de la plataforma, seguido de la parte numérica del
    show_id para asociarlo con el id de la información de los archivos de puntuación.
    """
    df['show_id'] = df['show_id'].str.replace('s' , platform + 's')
    df.rename({'show_id' : 'id'}, axis= 1, inplace=True)
    
    """
    Reemplaza Los valores nulos del campo rating por el string “G” (maturity rating: “general for all audiences”)
    """
    df = df.fillna({'rating' : 'G'})

    """
    Convierte el formato de fecha de la columna 'date_added' de: 'Mmmmmmm dd, AAAA' a: 'AAAA-mm-dd'
    """
    df.insert(6, 'AAAAmmdd', pd.to_datetime(df['date_added'].str.strip(), format='%B %d, %Y'), allow_duplicates=False)
    df['AAAAmmdd'] = df['AAAAmmdd'].astype('string')
    df['AAAAmmdd'] = df['AAAAmmdd'].str.slice(stop=10)
    df.drop(columns='date_added', inplace=True)
    df.rename({'AAAAmmdd' : 'date_added'}, axis= 1, inplace=True)

    """
    Genera dos columnas: duration_int y duration_type, apartir de la columna 'direction'. El primero será un integer y 
    el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)'
    """
    dur_split = df['duration'].str.split(expand=True)
    dur_split.columns = ['duration_int', 'duration_type']
    df = pd.concat([df, dur_split], axis=1)
    df = df.fillna({'duration_int' : '0'})
    df['duration_int'] = df['duration_int'].astype(np.int64)
    df['duration_type'] = df['duration_type'].str.replace('season' , 'seasons')
    df.drop(columns='duration', inplace=True)
    df = pd.DataFrame.reindex(df, columns = ['id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
                                             'release_year', 'rating', 'duration_int', 'duration_type', 
                                             'listed_in', 'description'])
    
    """
    Reemplaza Los valores nulos de las columnas 'director', 'cast', 'country', 'date_added', 'duration_type' 
    por el string 'sin dato'
    """
    #df = df.fillna({'director' : 'sin dato', 'cast' : 'sin dato', 'country' : 'sin dato',
    #                'date_added' : 'sin dato', 'duration_type' : 'sin dato'})
        
    """
    Convierte los datos de las columnas de tipo string a minusculas  
    """
    df['type'] = df['type'].str.lower()
    df['title'] = df['title'].str.lower()
    df['director'] = df['director'].str.lower()
    df['cast'] = df['cast'].str.lower()
    df['country'] = df['country'].str.lower()
    df['rating'] = df['rating'].str.lower()
    df['duration_type'] = df['duration_type'].str.lower()
    df['listed_in'] = df['listed_in'].str.lower()
    df['description'] = df['description'].str.lower()

    df.to_csv(out_file, index=False)
        
    return df

Ajusta los 4 archivos `csv` de plataformas-titulos.

In [74]:
file_in = r'D:\Henry\Repo\PP\datasets\netflix_titles.csv'
file_out = r'D:\Henry\Repo\PP\datasets\titles\netflix_titles_fixed.csv'
df_net_fix = fix_platform_file(file_in, 'n', file_out)

In [75]:
file_in = r'D:\Henry\Repo\PP\datasets\amazon_prime_titles.csv'
file_out = r'D:\Henry\Repo\PP\datasets\titles\amazon_prime_titles_fixed.csv'
df_ama_fix = fix_platform_file(file_in, 'a', file_out)

In [76]:
file_in = r'D:\Henry\Repo\PP\datasets\disney_plus_titles.csv'
file_out = r'D:\Henry\Repo\PP\datasets\titles\disney_plus_titles_fixed.csv'
df_dis_fix = fix_platform_file(file_in, 'd', file_out)

In [77]:
file_in = r'D:\Henry\Repo\PP\datasets\hulu_titles.csv'
file_out = r'D:\Henry\Repo\PP\datasets\titles\hulu_titles_fixed.csv'
df_hul_fix = fix_platform_file(file_in, 'h', file_out)

Genera un dataframe y un archivo consolidado de plataformas-titulos

In [78]:
df_titles, df_titles_len = join_csv(path_in=r'D:\Henry\Repo\PP\datasets\titles', 
                                    path_out=r'D:\Henry\Repo\PP\datasets\titles\join',
                                    out_file='platforms_titles.csv')
df_titles['id'] = df_titles['id'].astype('string')
df_titles['type'] = df_titles['type'].astype('string')
df_titles['title'] = df_titles['title'].astype('string')
df_titles['director'] = df_titles['director'].astype('string')
df_titles['cast'] = df_titles['cast'].astype('string')
df_titles['country'] = df_titles['country'].astype('string')
df_titles['date_added'] = df_titles['date_added'].astype('string')
df_titles['rating'] = df_titles['rating'].astype('string')
df_titles['duration_type'] = df_titles['duration_type'].astype('string')
df_titles['listed_in'] = df_titles['listed_in'].astype('string')
df_titles['description'] = df_titles['description'].astype('string')

In [79]:
df_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             22998 non-null  string
 1   type           22998 non-null  string
 2   title          22998 non-null  string
 3   director       14739 non-null  string
 4   cast           17677 non-null  string
 5   country        11499 non-null  string
 6   date_added     13444 non-null  string
 7   release_year   22998 non-null  int64 
 8   rating         22998 non-null  string
 9   duration_int   22998 non-null  int64 
 10  duration_type  22516 non-null  string
 11  listed_in      22998 non-null  string
 12  description    22994 non-null  string
dtypes: int64(2), string(11)
memory usage: 2.3 MB


Genera un dataframe consolidando plataformas, titulos y raitings.

In [81]:
df_tit_rat_merge = pd.merge(df_titles, df_ratings, on='id')

In [83]:
df_tit_rat_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024289 entries, 0 to 11024288
Data columns (total 16 columns):
 #   Column         Dtype  
---  ------         -----  
 0   id             string 
 1   type           string 
 2   title          string 
 3   director       string 
 4   cast           string 
 5   country        string 
 6   date_added     string 
 7   release_year   int64  
 8   rating         string 
 9   duration_int   int64  
 10  duration_type  string 
 11  listed_in      string 
 12  description    string 
 13  userId         int64  
 14  score          float64
 15  score_date     string 
dtypes: float64(1), int64(3), string(12)
memory usage: 1.4 GB


In [84]:
df_tit_rat_merge.head(3)

,id,type,title,director,cast,country,date_added,release_year,rating,duration_int,duration_type,listed_in,description,userId,score,score_date
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,"comedy, drama",a small fishing village must procure a local d...,543,5.0,2003-07-30
1,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,"comedy, drama",a small fishing village must procure a local d...,595,3.0,1996-08-13
2,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,"comedy, drama",a small fishing village must procure a local d...,611,3.0,2001-01-03


In [62]:
dfx = df_tit_rat_merge[(df_tit_rat_merge['id'] == 'as1') & (df_tit_rat_merge['title'] == 'the grand seduction')]
dfx.head(2)

,id,type,title,director,cast,country,date_added,release_year,rating,duration_int,duration_type,listed_in,description,userId,score,score_date
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,"comedy, drama",a small fishing village must procure a local d...,543,5.0,2003-07-30
1,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,"comedy, drama",a small fishing village must procure a local d...,595,3.0,1996-08-13


Genera un archivo `csv` consolidado de plataformas, titulos y raitings.

In [85]:
df_tit_rat_merge.to_csv(r'D:\Henry\Repo\PP\datasets\titles\join\platforms_titles_ratings.csv', index=False)

Genera un archivo `csv` consolidado de plataformas, titulos y raitings promedio. 

In [86]:
#df_tit_rat_mean = df_tit_rat_merge.groupby(['id', 'title', 'release_year']).score.mean()
df_tit_rat_mean = (df_tit_rat_merge.groupby(['id', 'title', 'date_added', 'release_year'], dropna=False)
                                   .agg({'score' : 'mean'}).reset_index())
df_tit_rat_mean['score'] = df_tit_rat_mean['score'].round(decimals=1)

df_tit_rat_mean.to_csv(r'D:\Henry\Repo\PP\datasets\titles\join\platforms_titles_ratings_m.csv', index=False)

In [87]:
df_tit_rat_mean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            22998 non-null  string 
 1   title         22998 non-null  string 
 2   date_added    13444 non-null  string 
 3   release_year  22998 non-null  int64  
 4   score         22998 non-null  float64
dtypes: float64(1), int64(1), string(3)
memory usage: 898.5 KB


Función que obtiene el número de títulos por plataforma.

In [152]:
def get_count_platform(platform):
    """
    Obtiene el número de títulos por plataforma.
    
    Parameters
    ----------
    platform : string
        Identificador de la plataforma de streaming.
    
    Returns
    -------
    int
        Número de títulos de la plataforma {'ama','dis','hel','net'}.
    """
    if platform == 'ama':
        p_id = 'as'
    elif platform == 'dis':
        p_id = 'ds'
    elif platform == 'hul':
        p_id = 'hs'
    else:
        p_id = 'ns'
    #df_plat = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTStV5K8QAKcxAEZsq8xLQiwZPK0X4gBxLLrpNQFhmjsXFgiRcDUWu3OtMapHOr3KcnomiM1hHV3NCD/pub?output=csv')
    df_plat = pd.read_csv(r'D:\Henry\Repo\PP\datasets\titles\join\platforms_titles.csv')
        
    df_plat = df_plat.loc[df_plat['id'].str.contains(p_id)]

    return(len(df_plat))

In [153]:
print(f'Cantidad de títulos: ', get_count_platform('ama'))

Cantidad de títulos:  9668


Función que obtiene el título de mayor duración de acuerdo a la plataforma, tipo y año.

In [108]:
def get_max_duration (platform, duration_type, year):
    """
    Obtiene el título de mayor duración de acuerdo a la plataforma, tipo y año.
    
    Parameters
    ----------
    platform : string
        Identificador de la plataforma de streaming {'ama','dis','hul','net'}.
    duration_type : string
        tipo de duración, {'min', 'season}.
    year : int
        Año de lanzamiento.
    
    Returns
    -------
    str
        Nombre de la película/serie de mayor duración.
    """
    if platform == 'ama':
        p_id = 'as'
    elif platform == 'dis':
        p_id = 'ds'
    elif platform == 'hul':
        p_id = 'hs'
    else:
        p_id = 'ns'
    df_plat = pd.read_csv(r'D:\Henry\Repo\PP\datasets\titles\join\platforms_titles.csv')
    df_plat = df_plat.loc[(df_plat['id'].str.contains(p_id)) & 
                          (df_plat['release_year'] == year)  &
                          (df_plat['duration_type'] == duration_type)]
    if len(df_plat) == 0:
        return('No hay títulos para la plataforma-tipo-año solicitados')
    
    max_dur = df_plat['duration_int'].max()
    df_tit = df_plat.loc[df_plat['duration_int'] == max_dur]
    title = str(df_tit['title'].values[0])
    
    return(title)

In [150]:
print(f'Título de mayor duración: ', get_max_duration('ama', 'seasons', 1986))

Título de mayor duración:  diff'rent strokes


Función que obtiene la cantidad de títulos por plataforma con un puntaje mayor a XX en determinado año. 

In [117]:
def get_score_count(platform, year, scored):
    """
    Obtiene la cantidad de títulos por plataforma con un puntaje mayor a XX en determinado año.
    
    Parameters
    ----------
    platform : string
        Identificador de la plataforma de streaming {'ama','dis','hul','net'}.
    year : int
        Año de lanzamiento.
    scored : float
        puntaje registrado.

    Returns
    -------
    int
        Número de títulos.
    """
    if platform == 'ama':
        p_id = 'as'
    elif platform == 'dis':
        p_id = 'ds'
    elif platform == 'hul':
        p_id = 'hs'
    else:
        p_id = 'ns'
    df_plat = pd.read_csv(r'D:\Henry\Repo\PP\datasets\titles\join\platforms_titles_ratings_m.csv')
    df_plat = df_plat[(df_plat['id'].str.contains(p_id)) & 
                      (df_plat['release_year'] == year)  & 
                      (df_plat['score'] > scored)]
    
    return(len(df_plat))

In [142]:
print(f'Cantidad de títulos: ', get_score_count('ama', 1920, 3.5))

Cantidad de títulos:  1


Función que obtiene el actor que más se repite por plataforma y año.

In [137]:
def get_actor(platform, year):
    """
    Obtiene el actor que más se repite por plataforma y año.
    
    Parameters
    ----------
    platform : string
        Identificador de la plataforma de streaming {'ama','dis','hul','net'}.
    year : int
        Año de lanzamiento.

    Returns
    -------
    string
        Actor que más se repite.
    """
    if platform == 'ama':
        p_id = 'as'
    elif platform == 'dis':
        p_id = 'ds'
    elif platform == 'hul':
        p_id = 'hs'
    else:
        p_id = 'ns'
    df_plat = pd.read_csv(r'D:\Henry\Repo\PP\datasets\titles\join\platforms_titles.csv')
    df_plat = df_plat[(df_plat['id'].str.contains(p_id)) & 
                      (df_plat['release_year'] == year)]
    if len(df_plat) == 0:
        return('No hay títulos para la plataforma-año solicitados')
    
    df_cast = df_plat['cast']
    actors = []
    for i in range(0, len(df_cast)):
        cast_tit = df_cast.iloc[i].split(',')
        for j in range(0, len(cast_tit)):
            actors.append(cast_tit[j].strip())
            
    return(mode(actors))

In [158]:
print(f'Actor más repetido: ', get_actor('net' , 1985))

Actor más repetido:  luis brandoni
